In [ ]:
# -*- coding: utf-8 -*-
"""Heart_failure_prediction.ipynb

Automatically generated by Colaboratory.

Original file is located at
    
"""

import pandas as pd
import numpy as np
import sklearn as skt
from fancyimpute import KNN
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

df = pd.read_csv("/content/drive/MyDrive/All datasets/heart_failure_clinical_records_dataset.csv")
df

df.dtypes

df.describe()

df.isnull().sum()

g=sns.countplot(df.DEATH_EVENT)

df.keys()

hf=df.copy()

cat_names=['anaemia','diabetes','high_blood_pressure','sex', 'smoking','DEATH_EVENT']

for i in cat_names:
  print(i)
  hf.loc[:,i]=hf.loc[:,i].round()
  hf.loc[:,i]=hf.loc[:,i].astype('object')

hf.dtypes

lis = []
for i in range(0,hf.shape[1]):
  #print(i)
  if(hf.iloc[:,i].dtype=="object"):
    hf.iloc[:,i]=pd.Categorical(hf.iloc[:,i])
    hf.iloc[:,i]=hf.iloc[:,i].cat.codes
    lis.append(hf.columns[i])

hf.head(5)

# outlier analysis ( Detecting & replacing )
hf2=hf.copy()

cnames=['age','creatinine_phosphokinase','ejection_fraction', 'platelets','serum_creatinine', 'serum_sodium','time']

for i in cnames:
  print(i)
  q75,q25=np.percentile(hf2.loc[:,i],[75,25])
  iqr=q75-q25
  min=q25-(1.5*iqr)
  max=q75+(1.5*iqr)
  print(min)
  print(max)
  hf2=hf2.replace((hf2[hf2.loc[:,i]<min].index),np.nan)
  hf2=hf2.replace((hf2[hf2.loc[:,i]>max].index),np.nan)

hf2.head(5)

hf2.isnull().sum()

missing_values2=pd.DataFrame(hf2.isnull().sum())
missing_values2

missing_values2 = missing_values2.reset_index()

missing_values2.columns = ["variables","missing_sum"]

missing_values2["missing%"]=missing_values2["missing_sum"]/len(hf2)
missing_values2.sort_values("missing%",axis=0,ascending=False).head(5)

# missing_value_imputation

hf2=pd.DataFrame(KNN(k=3).fit_transform(hf2),columns=hf2.columns)

hf2.isnull().sum()

for i in lis:
  hf2.loc[:,i]=hf2.loc[:,i].round()
  hf2.loc[:,i]=hf2.loc[:,i].astype('object')

hf3=hf2.copy()

# Checking_multicolinarity with Corelation

#cnames=['age','creatinine_phosphokinase','ejection_fraction', 'platelets','serum_creatinine', 'serum_sodium','time']

hf3_corr = hf3.loc[:,cnames]
corr = hf3_corr.corr()
highly_corelated = corr[corr>0.95]
highly_corelated

#feature Selection_VIF

from statsmodels.stats.outliers_influence import variance_inflation_factor

hf3_VIF = hf3.loc[:,cnames]
vif_data = pd.DataFrame()
vif_data["feature"] = hf3_VIF.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(hf3_VIF.values, i)
                          for i in range(len(hf3_VIF.columns))]

print(vif_data)

# Removing Categorical Variables
#cat_names=['anaemia','diabetes','high_blood_pressure','sex', 'smoking','DEATH_EVENT']

for i in cat_names:
  print(i)
  chi2,p,dof,ex = chi2_contingency(pd.crosstab(hf3.DEATH_EVENT,hf3[i]))
  print(chi2,p,dof,ex)

# Dropping Columns with corelation & high P value

hf3=hf3.drop(['age','ejection_fraction','platelets','serum_sodium','anaemia','diabetes','high_blood_pressure','sex','smoking'],axis=1)

hf3.head(5)

hf4=hf3.copy()

#normality check
plt.hist(hf4.creatinine_phosphokinase,bins="auto")

hf4 = hf4.drop(['DEATH_EVENT'],axis=1)

#Normalisation
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
hf4= hf4.values #returns a numpy array
x_scaled = min_max_scaler.fit_transform(hf4)
hf4 = pd.DataFrame(x_scaled)
hf4.head(5)

hf4.columns = ['creatinine_phosphokinase','serum_creatinine','time']

hf4.keys()

hf4['DEATH_EVENT']=hf3['DEATH_EVENT']
hf4.shape

#sampling
g = sns.countplot(hf4.DEATH_EVENT)
plt.show()

# Oversampling

class_count_0,class_count_1=hf4['DEATH_EVENT'].value_counts()
print(class_count_0,class_count_1)
class_0 = hf4[hf4['DEATH_EVENT'] == 0]
class_1 = hf4[hf4['DEATH_EVENT'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

# Data is limited_Use Oversampling

class_1_over = class_1.sample(class_count_0,replace=True)
class_1_over.shape
test_over = pd.concat([class_1_over,class_0],axis=0)

print(test_over['DEATH_EVENT'].value_counts())

g1 = sns.countplot(test_over['DEATH_EVENT'])
plt.show()

hf5 = test_over.copy()

hf5.keys()

from sklearn.model_selection import train_test_split

# Model Development

X_over = hf5.values[:,0:3]
Y_over = hf5.values[:,3]
Y_over=Y_over.astype('int')
X_over_train,X_over_test,Y_over_train,Y_over_test = train_test_split(X_over,Y_over, test_size = 0.3, random_state = 42)

# logistic regression
from sklearn.linear_model import LogisticRegression
lr_over = LogisticRegression()

logit_model= lr_over.fit(X_over_train,Y_over_train)

Y_Over_Pred = lr_over.predict(X_over_test)

from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(recall_score(Y_over_test,Y_Over_Pred))
print(accuracy_score(Y_over_test,Y_Over_Pred))
print(confusion_matrix(Y_over_test,Y_Over_Pred))

CM_Crosstab = pd.crosstab(Y_over_test,Y_Over_Pred)
CM_Crosstab.columns=["No","Yes"]
CM_Crosstab.index = ["No","Yes"]
CM_Crosstab

FNR = (CM_Crosstab.iloc[1,0]/(CM_Crosstab.iloc[1,1]+CM_Crosstab.iloc[1,0]))*100
FNR

from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics
fpr, tpr, thresholds = roc_curve(Y_over_test,Y_Over_Pred)
roc_auc = metrics.auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Logistic Regression')
display.plot()  
plt.show()

# Decision Tree
from sklearn import tree
dt = tree.DecisionTreeClassifier(criterion="entropy",max_depth=3, random_state=1234) #DecisionTreeClassifier(max_depth=3, random_state=1234)

c50_model = dt.fit(X_over_train,Y_over_train)
Y_Over_Pred_C50 = dt.predict(X_over_test)

from sklearn import tree

#clf = tree.DecisionTreeClassifier(max_leaf_nodes=n)
#clf_ = clf.fit(X, data_y)
X_over_train_1=pd.DataFrame(X_over_train)
X_over_train_1.columns = ['creatinine_phosphokinase', 'serum_creatinine', 'time']
feature_names = X_over_train_1.columns
class_name = c50_model.classes_.astype(int).astype(str)

def output_pdf(c50_model, name):
    from sklearn import tree
    from sklearn.externals.six import StringIO
    import pydot_ng as pydot
    dot_data = StringIO()
    tree.export_graphviz(c50_model, out_file=dot_data,
                         feature_names=feature_names,
                         class_names=class_name,
                         filled=True, rounded=True,
                         special_characters=True,
                          node_ids=1,)
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    graph.write_pdf("%s.pdf"%name)

output_pdf(c50_model, name='filename%s')

print(recall_score(Y_over_test,Y_Over_Pred_C50))
print(accuracy_score(Y_over_test,Y_Over_Pred_C50))
confusion_matrix(Y_over_test,Y_Over_Pred_C50)

CM_Crosstab_1 = pd.crosstab(Y_over_test,Y_Over_Pred_C50)
CM_Crosstab_1.columns=["No","Yes"]
CM_Crosstab_1.index = ["No","Yes"]
CM_Crosstab_1

FNR = (CM_Crosstab_1.iloc[1,0]/(CM_Crosstab_1.iloc[1,1]+CM_Crosstab_1.iloc[1,0]))*100
FNR

from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics
fpr, tpr, thresholds = roc_curve(Y_over_test,Y_Over_Pred_C50)
roc_auc = metrics.auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Decision Tree')
display.plot()  
plt.show()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500)

rf_model = rf.fit(X_over_train,Y_over_train)
Y_Over_Pred_rf = rf.predict_proba(X_over_test) # predict_proba for predicting probalities

Y_Over_Pred_rf_1=pd.DataFrame(Y_Over_Pred_rf)

Y_Over_Pred_rf_1.head(5)

Y_over_test_1=pd.DataFrame(Y_over_test)

Y_over_test_1.head(5)

#result = left.join(right)
Y_NEW = pd.concat([Y_over_test_1,Y_Over_Pred_rf_1],axis=1)

Y_NEW.head(5)

rf_model = rf.fit(X_over_train,Y_over_train)
Y_Over_Pred_rff = rf.predict(X_over_test)
print(recall_score(Y_over_test,Y_Over_Pred_rff))
print(accuracy_score(Y_over_test,Y_Over_Pred_rff))

CM_Crosstab_2 = pd.crosstab(Y_over_test,Y_Over_Pred_rff)
CM_Crosstab_2.columns=["No","Yes"]
CM_Crosstab_2.index = ["No","Yes"]
CM_Crosstab_2

FNR = (CM_Crosstab_2.iloc[1,0]/(CM_Crosstab_2.iloc[1,1]+CM_Crosstab_2.iloc[1,0]))*100
FNR

from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_over_test,Y_Over_Pred))
print(roc_auc_score(Y_over_test,Y_Over_Pred_C50))
print(roc_auc_score(Y_over_test,Y_Over_Pred_rff))

from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics
fpr, tpr, thresholds = roc_curve(Y_over_test,Y_Over_Pred_rff)
roc_auc = metrics.auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='RandomForest')
display.plot()  
plt.show()